Provided by Arshy from Icevision Discord

In [1]:
# !wget https://raw.githubusercontent.com/airctic/icevision/master/install_colab.sh
# !bash install_colab.sh

In [ ]:
#!pip install effdet -q

In [1]:
import fastai, icevision, torch
print(fastai.__version__, icevision.__version__, torch.__version__)

2.3.1 0.8.0 1.8.0+cu111


In [2]:
import pandas as pd
import timm
import torch
from torch import tensor
from torch import nn
from pathlib import Path
from collections import namedtuple 

In [3]:
from icevision.core import ClassMap
from icevision.core.bbox import BBox
from icevision.core.tasks import Task
from icevision.models.ross.efficientdet.dataloaders import process_train_record, build_infer_batch
from icevision.models.ross.efficientdet import convert_raw_predictions
from icevision.core.record import BaseRecord
from icevision.core.record_components import (FilepathRecordComponent, InstancesLabelsRecordComponent, 
                                              BBoxesRecordComponent, ClassificationLabelsRecordComponent, 
                                              ScoresRecordComponent)
from icevision.parsers.parser import Parser
from icevision.data.dataset import Dataset
from icevision import tfms
from icevision.models.utils import transform_dl
from icevision.metrics.coco_metric import COCOMetricType, COCOMetric
from icevision.models.utils import apply_batch_tfms, unload_records

In [4]:
from effdet import get_efficientdet_config, create_model_from_config, unwrap_bench, create_model, load_checkpoint
from effdet.bench import _post_process, _batch_detection
from effdet.config import set_config_readonly, set_config_writeable
from effdet.efficientdet import get_feature_info, BiFpn, BiFpnLayer, HeadNet, _init_weight, _init_weight_alt
from effdet.loss import DetectionLoss
from effdet.anchors import Anchors, AnchorLabeler

In [5]:
from fastai.tabular.core import TabularPandas, Categorify, FillMissing, Normalize
from fastai.tabular.model import get_emb_sz, TabularModel
from fastai.data.block import CategoryBlock
from fastai.vision.learner import create_head
from fastai.callback.hook import num_features_model
from fastai.callback.core import Callback
from fastai.optimizer import ranger
from fastai.losses import CrossEntropyLossFlat
from fastai.torch_core import params
from fastai.data.load import DataLoader

In [6]:
from fastcore.all import *

In [7]:
ImgSize = namedtuple("ImgSize", "width,height")

In [8]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

# Paths and preparing df

In [10]:

path = Path('/host_Data/DataSets/Kaggle-SIIM Covid 19/siim-covid_resize_1024px')
path_img = path/'train'

In [11]:
dcm_df = pd.read_csv(path/'COVID_dcm_metadata.csv')
train_df = pd.read_csv(path/'COVID_train_df.csv')
bb_df = pd.read_csv(path/'bounding_boxes256.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/host_Data/DataSets/Kaggle-SIIM Covid 19/siim-covid_resize_1024px/COVID_dcm_metadata.csv'

In [ ]:
train_df['image_id'] = train_df['id'].apply(lambda x: f'{x.split("_image")[0]}.jpg')
train_df['classi_label'] = train_df[['Negative for Pneumonia', 'Typical Appearance','Indeterminate Appearance', 'Atypical Appearance']].idxmax(1)
train_df = train_df[['image_id', 'classi_label']]

In [ ]:
dcm_df = dcm_df[['fname', 'Modality', 'PatientSex']]
dcm_df['image_id'] = dcm_df.fname.apply(lambda x: f'{x.split("/")[-1].split(".dcm")[0]}.jpg')
train_df = train_df.merge(dcm_df, on='image_id')

In [ ]:
df = bb_df.merge(train_df, on='image_id')
df['box_label'] = df['label'].apply(lambda x: x.split(' ')[0])

In [ ]:
df['bboxes'] = None
for i, row in df.iterrows():
    lab = row['label'].split(' ')
    new = []
    for j in range(len(lab)):
        if (lab[j] == 'opacity') or (lab[j] == 'none'):
            new.append(lab[j+2:j+6])
    df.loc[i, 'bboxes'] = new

In [ ]:
df['Modality_n'] = df['Modality'].map({'DX':0, 'CR':1})
df['PatientSex_n'] = df['PatientSex'].map({'M':0, 'F':1})

# Parser and dataloader

In [ ]:
# Here ObjectDetectionRecord from icevision and added ClassificationLabelsRecordComponent
def ObjectDetectionClassiRecord():
    return BaseRecord(
        (
            FilepathRecordComponent(),
            InstancesLabelsRecordComponent(),
            BBoxesRecordComponent(),
            ClassificationLabelsRecordComponent(),
            ScoresRecordComponent(task=Task('tabular')),
        )
    )
template_record = ObjectDetectionClassiRecord()

In [ ]:
#custom parser from icevision - added classification class map and classification labels
class CustomParser(Parser):
    def __init__(self, template_record, img_dir, label_df):
        super().__init__(template_record=template_record)

        self.img_dir = img_dir
        self.df = label_df
        self.class_map = ClassMap(list(self.df['box_label'].unique()))
        self.classi_class_map = ClassMap(list(self.df['classi_label'].unique()), background=None)

    def __iter__(self):
        for o in self.df.itertuples():
            yield o

    def __len__(self) -> int:
        return len(self.df)

    def record_id(self, o):
        return o.image_id

    def parse_fields(self, o, record, is_new):
        if is_new:
            record.set_filepath(f'{self.img_dir}/{o.image_id}')
            record.set_img_size(ImgSize(width=256, height=256))
            record.detection.set_class_map(self.class_map)
            record.classification.set_class_map(self.classi_class_map)
            record.classification.set_labels([o.classi_label])
            record.tabular.set_scores([o.Modality_n, o.PatientSex_n])

        record.detection.add_bboxes([BBox.from_xyxy(float(bb[0]), float(bb[1]), float(bb[2]), float(bb[3])) for bb in o.bboxes])
        record.detection.add_labels([o.box_label for _ in o.bboxes])

In [ ]:
parser = CustomParser(template_record, path_img, df)
train_records, valid_records = parser.parse(autofix=True)

  0%|          | 0/6334 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:136


  0%|          | 0/5067 [00:00<?, ?it/s]

  0%|          | 0/1267 [00:00<?, ?it/s]

In [ ]:
#TODO: to add only specific augmentations 
train_tfms = tfms.A.Adapter([*tfms.A.aug_tfms(size=256), tfms.A.Normalize()])
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(256), tfms.A.Normalize()])

In [ ]:
# Datasets
train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

In [ ]:
# the fuction below is extension of build_train_batch. this is to add batch_classi targets["classi"] is for targets for classification
def build_train_batch_m(records):

    batch_images, batch_bboxes, batch_classes = zip(
        *(process_train_record(record) for record in records)
    )
    batch_classi = [record.classification.label_ids for record in records] #INSERTED CODE
    batch_tabular = [record.tabular.scores for record in records] #INSERTED CODE

    # convert to tensors
    batch_images = torch.stack(batch_images)
    batch_bboxes = [tensor(bboxes, dtype=torch.float32) for bboxes in batch_bboxes]
    batch_classes = [tensor(classes, dtype=torch.float32) for classes in batch_classes] 
    batch_classi = [tensor(classes, dtype=torch.float32) for classes in batch_classi] #INSERTED CODE
    batch_tabular = [tensor(scores, dtype=torch.int32) for scores in batch_tabular] #INSERTED CODE

    # convert to EffDet interface
    targets = dict(bbox=batch_bboxes, cls=batch_classes, tab=batch_tabular, classi=batch_classi)

    return (batch_images, targets), records

In [ ]:
# the fuction below make use of build_train_batch_ to build valid_ds
def build_valid_batch_m(records):

    (batch_images, targets), records = build_train_batch_m(records)

    # convert to EffDet interface, when not training, dummy size and scale is required
    targets = dict(img_size=None, img_scale=None, **targets)

    return (batch_images, targets), records

In [ ]:
def transform_dl_m(dataset, build_batch, batch_tfms=None, **dataloader_kwargs):
    """Creates collate_fn from build_batch by decorating it with apply_batch_tfms and unload_records"""
    collate_fn = apply_batch_tfms(build_batch, batch_tfms=batch_tfms)
    collate_fn = unload_records(collate_fn)
    return DataLoader(dataset=dataset, create_batch=collate_fn, **dataloader_kwargs)

In [ ]:
def train_dl_m(dataset, batch_tfms=None, **dataloader_kwargs):
    return transform_dl_m(
        dataset=dataset,
        build_batch=build_train_batch_m,
        batch_tfms=batch_tfms,
        **dataloader_kwargs
    )

In [ ]:
def valid_dl_m(dataset, batch_tfms=None, **dataloader_kwargs):
    return transform_dl_m(
                        dataset=dataset,
                        build_batch=build_valid_batch_m,
                        batch_tfms=batch_tfms,
                        **dataloader_kwargs
                        )

In [ ]:
train_dl = train_dl_m(train_ds, batch_size=8, num_workers=2, shuffle=True)
valid_dl = valid_dl_m(valid_ds, batch_size=8, num_workers=2, shuffle=False)

In [ ]:
procs = [Categorify, FillMissing, Normalize]
cat_names  = ['Modality', 'PatientSex']

to = TabularPandas(train_df, procs, cat_names, y_names='classi_label', y_block=CategoryBlock())
emb_szs = get_emb_sz(to)

# Preparing DetClassiBench + tabular data embeddings

In [ ]:
#modeified effdet's EfficientDet to calculate classification output from backbone's output
class EfficientDetClassi(nn.Module):
    def __init__(self, config, pretrained_backbone=True, classi_class=None, emb_szs=emb_szs, alternate_init=False):
        super(EfficientDetClassi, self).__init__()
        self.config = config
        set_config_readonly(self.config)
        self.backbone = timm.create_model(
            config.backbone_name, features_only=True,
            out_indices=self.config.backbone_indices or (2, 3, 4),
            pretrained=pretrained_backbone, **config.backbone_args)
        feature_info = get_feature_info(self.backbone)
        self.fpn = BiFpn(self.config, feature_info)
        self.class_net = HeadNet(self.config, num_outputs=self.config.num_classes)
        self.box_net = HeadNet(self.config, num_outputs=4)

        #INSERTED CODE STARTS
        backbone_features = num_features_model(nn.Sequential(*self.backbone.children()))
        self.vis_head = create_head(backbone_features, 100) #fastai's create_head
        self.tab = TabularModel(emb_szs, n_cont=0, out_sz = 100, layers = [100, 250])
        self.final_head = nn.Sequential(
                                        nn.BatchNorm1d(200),
                                        nn.Dropout(0.25),
                                        nn.Linear(200, 100, bias=False),
                                        nn.ReLU(inplace=True),
                                        nn.BatchNorm1d(100),
                                        nn.Dropout(0.5),
                                        nn.Linear(100, 4)
                                            )  
        #INSERTED CODE ENDS
        
        '''
        self.classifier = nn.Sequential(
                                        nn.AdaptiveMaxPool2d(output_size=1),
                                        nn.Flatten(),
                                        nn.BatchNorm1d(backbone_features),
                                        nn.Dropout(p=0.25, inplace=False),
                                        nn.Linear(backbone_features, 512),
                                        nn.ReLU(inplace=True),
                                        nn.BatchNorm1d(512),
                                        nn.Dropout(p=0.25, inplace=False),
                                        nn.Linear(512, classi_class), 
        )
  
        '''
        for n, m in self.named_modules():
            if 'backbone' not in n:
                if alternate_init:
                    _init_weight_alt(m, n)
                else:
                    _init_weight(m, n)

    @torch.jit.ignore()
    def reset_head(self, num_classes=None, aspect_ratios=None, num_scales=None, alternate_init=False):
        reset_class_head = False
        reset_box_head = False
        set_config_writeable(self.config)
        if num_classes is not None:
            reset_class_head = True
            self.config.num_classes = num_classes
        if aspect_ratios is not None:
            reset_box_head = True
            self.config.aspect_ratios = aspect_ratios
        if num_scales is not None:
            reset_box_head = True
            self.config.num_scales = num_scales
        set_config_readonly(self.config)

        if reset_class_head:
            self.class_net = HeadNet(self.config, num_outputs=self.config.num_classes)
            for n, m in self.class_net.named_modules(prefix='class_net'):
                if alternate_init:
                    _init_weight_alt(m, n)
                else:
                    _init_weight(m, n)

        if reset_box_head:
            self.box_net = HeadNet(self.config, num_outputs=4)
            for n, m in self.box_net.named_modules(prefix='box_net'):
                if alternate_init:
                    _init_weight_alt(m, n)
                else:
                    _init_weight(m, n)

    @torch.jit.ignore()
    def toggle_head_bn_level_first(self):
        """ Toggle the head batchnorm layers between being access with feature_level first vs repeat
        """
        self.class_net.toggle_bn_level_first()
        self.box_net.toggle_bn_level_first()

    def forward(self, x, tab):
        x_b = self.backbone(x)
        x = self.fpn(x_b)
        x_class = self.class_net(x)
        x_box = self.box_net(x)
        x_classi_vis = self.vis_head(x_b[2])   #INSERTED CODE
        x_class_tab = self.tab(tab)            #INSERTED CODE
        x_classi = self.final_head(torch.cat([x_class_tab, x_classi_vis], dim=1))
        return x_class, x_box, x_classi #returns x_classi on top of original

In [ ]:
# Used DetBenchTrain  
class DetClassiBenchTrain(nn.Module):
    def __init__(self, model, create_labeler=True):
        super(DetClassiBenchTrain, self).__init__()
        self.model = model
        self.config = model.config
        self.num_levels = model.config.num_levels
        self.num_classes = model.config.num_classes
        self.anchors = Anchors.from_config(model.config)
        self.max_detection_points = model.config.max_detection_points
        self.max_det_per_image = model.config.max_det_per_image
        self.soft_nms = model.config.soft_nms
        self.anchor_labeler = None
        if create_labeler:
            self.anchor_labeler = AnchorLabeler(self.anchors, self.num_classes, match_threshold=0.5)
        self.loss_fn = DetectionLoss(model.config)
        self.classi_loss = CrossEntropyLossFlat() #INSERTED CODE - loss_fn for classification loss

    def forward(self, x, target):
        tab = torch.stack(target['tab']).type(torch.LongTensor).to('cuda')
        class_out, box_out, classi_out = self.model(x, tab) #OUTPUTS INCLUDE classiicaiton predictions
        if self.anchor_labeler is None:
            # target should contain pre-computed anchor labels if labeler not present in bench
            assert 'label_num_positives' in target
            cls_targets = [target[f'label_cls_{l}'] for l in range(self.num_levels)]
            box_targets = [target[f'label_bbox_{l}'] for l in range(self.num_levels)]
            num_positives = target['label_num_positives']
        else:
            cls_targets, box_targets, num_positives = self.anchor_labeler.batch_label_anchors(
                target['bbox'], target['cls'])

        classi_targets = torch.stack(target['classi']).squeeze().type(torch.LongTensor).to('cuda')
        loss, class_loss, box_loss = self.loss_fn(class_out, box_out, cls_targets, box_targets, num_positives)
        
        #INSERTED CODE STARTS
        classi_loss = self.classi_loss(classi_out, classi_targets)
        loss += classi_loss
        #INSERTED CODE ENDS

        output = {'loss': loss, 'class_loss': class_loss, 'box_loss': box_loss, 'classi_pred':classi_out, 'classi_loss':classi_loss} #ADDED "classi_loss"

        if not self.training:
            # if eval mode, output detections for evaluation
            class_out_pp, box_out_pp, indices, classes = _post_process(
                class_out, box_out, num_levels=self.num_levels, num_classes=self.num_classes,
                max_detection_points=self.max_detection_points)
            output['detections'] = _batch_detection(
                x.shape[0], class_out_pp, box_out_pp, self.anchors.boxes, indices, classes,
                target['img_scale'], target['img_size'],
                max_det_per_image=self.max_det_per_image, soft_nms=self.soft_nms)
            
        return output

In [ ]:
def create_model_m(model_name, bench_task='', num_classes=None, pretrained=False,
                 checkpoint_path='', checkpoint_ema=False, img_size=None, **kwargs):

    config = get_efficientdet_config(model_name)
    config.image_size = (img_size, img_size) if isinstance(img_size, int) else img_size

    return create_model_from_config_m(config, bench_task=bench_task, num_classes=num_classes, pretrained=pretrained,
                                      checkpoint_path=checkpoint_path, checkpoint_ema=checkpoint_ema, **kwargs)

In [ ]:
def create_model_from_config_m(
        config, bench_task='', num_classes=None, pretrained=False,
        checkpoint_path='', checkpoint_ema=False, **kwargs):

    pretrained_backbone = kwargs.pop('pretrained_backbone', True)
    if pretrained or checkpoint_path:
        pretrained_backbone = False  # no point in loading backbone weights

    # Config overrides, override some config values via kwargs.
    overrides = (
        'redundant_bias', 'label_smoothing', 'legacy_focal', 'jit_loss', 'soft_nms', 'max_det_per_image', 'image_size')
    for ov in overrides:
        value = kwargs.pop(ov, None)
        if value is not None:
            setattr(config, ov, value)

    labeler = kwargs.pop('bench_labeler', False)

    # create the base model
    model = EfficientDetClassi(config, pretrained_backbone=pretrained_backbone, **kwargs)
    
    # pretrained weights are always spec'd for original config, load them before we change the model
    if pretrained:
        load_pretrained(model, config.url)

    # reset model head if num_classes doesn't match configs
    if num_classes is not None and num_classes != config.num_classes:
        model.reset_head(num_classes=num_classes)

    # load an argument specified training checkpoint
    if checkpoint_path:
        load_checkpoint(model, checkpoint_path, use_ema=checkpoint_ema)

    # wrap model in task specific training/prediction bench if set
    if bench_task == 'train':
        model = DetClassiBenchTrain(model, create_labeler=True)
    elif bench_task == 'predict':
        model = DetClassiBenchPredict(model)
    return model

In [ ]:
def splitter(m):
    s = nn.Sequential(*m.model.children())
    return L(s[0], s[1:]).map(params)

In [ ]:
#lazy hack to report classification accuracy
class ClassiAccuracy(Callback):
    def after_epoch(self):
        self.learn.eval()
        acc, count = 0, 0
        for xv, yv in self.learn.dls.valid:
            out = self.learn.model(xv[0], xv[1])
            pred = torch.argmax(out["classi_pred"], 1).detach().to("cpu")
            targ = torch.stack(xv[1]["classi"]).squeeze().type(torch.LongTensor).detach().to('cpu')
            acc += (pred == targ).float().sum()
            count += xv[0].shape[0]
            
        print(f'Epoch {self.learn.epoch} : {acc/count*100:.3}')

In [ ]:
bench = create_model_m('tf_efficientdet_d3', 
                        'train',  
                        num_classes=len(parser.class_map)-1,
                        classi_class=len(parser.classi_class_map),
                        img_size=256)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b3_aa-84b4657e.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b3_aa-84b4657e.pth


In [ ]:
model_type = icevision.models.ross.efficientdet
metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]
learn = model_type.fastai.learner(dls=[train_dl, valid_dl], 
                                  model=bench, 
                                  metrics=metrics,
                                  splitter = splitter,
                                  opt_func = ranger)

In [ ]:
learn.freeze()

In [ ]:
learn.fit_flat_cos(1, 1e-3)

epoch,train_loss,valid_loss,COCOMetric,time
0,2.201867,2.145974,0.025455,14:39


In [ ]:
learn.fit_flat_cos(3, 1e-3, cbs=ClassiAccuracy())

epoch,train_loss,valid_loss,COCOMetric,time
0,2.000117,2.000911,0.046562,03:53
1,1.884191,1.994362,0.054217,03:53
2,1.846415,1.860683,0.066336,03:54


Epoch 0 : 60.4
Epoch 1 : 60.5
Epoch 2 : 62.2


In [ ]:
learn.unfreeze()
learn.fit_flat_cos(15, 1e-3, cbs=ClassiAccuracy())

NameError: ignored